Creating standard folders for dataset :

In [ ]:
!rm -r /content/dataset
!mkdir /content/dataset

!mkdir /content/dataset/test
!mkdir /content/dataset/train
!mkdir /content/dataset/valid

!mkdir /content/dataset/test/images
!mkdir /content/dataset/test/labels

!mkdir /content/dataset/train/images
!mkdir /content/dataset/train/labels

!mkdir /content/dataset/valid/images
!mkdir /content/dataset/valid/labels

Reducing image size on memory :

In [ ]:
image_format = '.jpg'

Copying training images in dataset folder:

In [ ]:
import os
import cv2
train_img_path = '/content/data/RawImage/TrainingData/'
destination_path = '/content/dataset/train/images/'
# Iterate directory
for filename in os.listdir(train_img_path):
  image = cv2.imread(train_img_path + filename)
  cv2.imwrite('/content/dataset/train/images/'+filename.replace('.bmp', image_format), image)

Copying testing images in dataset folder:

In [ ]:
import os
import cv2
test_img_path = '/content/data/RawImage/Test1Data/'
destination_path = '/content/dataset/test/images/'
# Iterate directory
for filename in os.listdir(test_img_path):
  image = cv2.imread(test_img_path + filename)
  cv2.imwrite('/content/dataset/test/images/'+filename.replace('.bmp', image_format), image)

Copying validation images in dataset folder:

In [ ]:
import os
import cv2
valid_img_path = '/content/data/RawImage/Test2Data/'
destination_path = '/content/dataset/valid/images/'
# Iterate directory
for filename in os.listdir(valid_img_path):
  image = cv2.imread(valid_img_path + filename)
  cv2.imwrite('/content/dataset/valid/images/'+filename.replace('.bmp', image_format), image)

Checking images size in train folder:

In [ ]:
from PIL import Image

for i in range(1,151,1):
  sourceimage = '/content/dataset/train/images/' + '{0:03}'.format(i)+image_format
  image = Image.open(sourceimage)
  print(image.size)
print('done')

Defining some variables :

In [ ]:
img_width = 1935
img_height = 2400
bb_margin = 25

Converting point to bounding box :

In [ ]:
def xy_to_bbox(xy, margin = 10, width = img_width, height = img_height):
  x = int(xy[0])
  y = int(xy[1])
  return [str(x/width), str(y/height), str((2*margin)/width), str((2*margin)/height)]

Copying training annotation in dataset folder:

In [ ]:
for i in range(1,151,1):
  sourcefile = '/content/data/annotations/400_senior/' + '{0:03}'.format(i)+'.txt'
  destinationfile = '/content/dataset/train/labels/' + '{0:03}'.format(i)+'.txt'

  fp = open(destinationfile, 'w')
  with open(sourcefile) as myfile:
    for x in range(19):
      content = ''
      line = (next(myfile).strip().split(','))
      line = xy_to_bbox(line, margin = bb_margin, width = img_width, height = img_height)
      content = str(x)
      for item in line:
        content = content + ' ' + item
      # write each item on a new line
      fp.write("%s\n" % content)

Copying testing annotation in dataset folder:

In [ ]:
for i in range(151,301,1):
  sourcefile = '/content/data/annotations/400_senior/' + '{0:03}'.format(i)+'.txt'
  destinationfile = '/content/dataset/test/labels/' + '{0:03}'.format(i)+'.txt'

  fp = open(destinationfile, 'w')
  with open(sourcefile) as myfile:
    for x in range(19):
      content = ''
      line = (next(myfile).strip().split(','))
      line = xy_to_bbox(line, margin = bb_margin, width = img_width, height = img_height)
      content = str(x)
      for item in line:
        content = content + ' ' + item
      # write each item on a new line
      fp.write("%s\n" % content)

Copying validation annotation in dataset folder:

In [ ]:
for i in range(301,401,1):
  sourcefile = '/content/data/annotations/400_senior/' + '{0:03}'.format(i)+'.txt'
  destinationfile = '/content/dataset/valid/labels/' + '{0:03}'.format(i)+'.txt'

  fp = open(destinationfile, 'w')
  with open(sourcefile) as myfile:
    for x in range(19):
      content = ''
      line = (next(myfile).strip().split(','))
      line = xy_to_bbox(line, margin = bb_margin, width = img_width, height = img_height)
      content = str(x)
      for item in line:
        content = content + ' ' + item
      # write each item on a new line
      fp.write("%s\n" % content)

----------------------------------------
Defining some variables :

In [ ]:
img_width = 1935
img_height = 2400
augmentation_margin = 1
bb_margin = 25

Installing image augmentation package :

In [ ]:
!pip install imgaug

Writing bounding box detailes for augmented images in annotation file in dataset folder :

In [ ]:
def write_annotation_to_file_augmentation(ih, iw, bbs, file_name):
  fp = open(file_name, 'w')
  for x in range(19):
      content = ''
      content = str(x)
      content = content + ' ' + str(bbs[x][0]/iw) + ' ' + str(bbs[x][1]/ih) \
                        + ' ' + str(bbs[x][2]/iw) + ' ' + str(bbs[x][3]/ih)
      
      # write each item on a new line
      fp.write("%s\n" % content)


Converting point to bounding box :

In [ ]:
def xy_to_bbox_augmentation(xy, margin = 5, width = img_width, height = img_height):
  x = int(xy[0])
  y = int(xy[1])
  return [(x - margin), (y - margin), (x + margin), (y + margin)]

Converting augmented bbs to yolo specified format :

In [ ]:
def convert_bbox_to_yolo_format(bbs, margin = 10, width = img_width, height = img_height):
  bb = []
  for i in range(len(bbs)):
    x = ((bbs[i][0][0] + bbs[i][1][0])/2)
    y = ((bbs[i][0][1] + bbs[i][1][1])/2)
    bb.append([x, y, (2*margin), (2*margin)])
  return bb

Function for applying 11 deferent augmentation method on each image :

In [ ]:
import imgaug as ia
import imgaug.augmenters as iaa
import imgaug.parameters as iap
import cv2

aug1 = iaa.AdditiveLaplaceNoise(scale=0.1*255)
aug2 = iaa.MultiplyElementwise((0.5, 1.5))
aug3 = iaa.CoarseDropout((0.0, 0.05), size_percent=(0.02, 0.25))
aug4 = iaa.CoarseSaltAndPepper(0.05, size_percent=(0.01, 0.1))
aug5 = iaa.BlendAlphaSimplexNoise(iaa.EdgeDetect(1.0), sigmoid_thresh=iap.Normal(10.0, 5.0))
aug6 = iaa.SigmoidContrast(gain=(3, 10), cutoff=(0.4, 0.6))
aug7 = iaa.AllChannelsCLAHE()
aug8 = iaa.Flipud(1.0)
aug9 = iaa.Affine(rotate=(-45, 45))
aug10 = iaa.PiecewiseAffine(scale=(0.01, 0.05))
aug11 = iaa.Rot90([1, 3])

def aug(image2, bbs):
  images_aug, bbs_aug = aug1(images=image2, bounding_boxes=bbs)
  bbs_aug = convert_bbox_to_yolo_format(bbs_aug, margin = bb_margin, width = img_width, height = img_height)
  res = cv2.cvtColor(images_aug[0], cv2.COLOR_RGB2BGR)
  img_file_name = destinationimagePath + '1_' + image_name
  cv2.imwrite(img_file_name, res)
  ann_file_name = destinationannotationPath + '1_'+ image_name.replace(image_format, '.txt')
  write_annotation_to_file_augmentation(ih, iw, bbs_aug, ann_file_name)
  print(i, ': 1', end = ' ')
  
  images_aug, bbs_aug = aug2(images=image2, bounding_boxes=bbs)
  bbs_aug = convert_bbox_to_yolo_format(bbs_aug, margin = bb_margin, width = img_width, height = img_height)
  res = cv2.cvtColor(images_aug[0], cv2.COLOR_RGB2BGR)
  img_file_name = destinationimagePath + '2_'+image_name
  cv2.imwrite(img_file_name, res)
  ann_file_name = destinationannotationPath + '2_'+image_name.replace(image_format, '.txt')
  write_annotation_to_file_augmentation(ih, iw, bbs_aug, ann_file_name)
  print('2', end = ' ')

  images_aug, bbs_aug = aug3(images=image2, bounding_boxes=bbs)
  bbs_aug = convert_bbox_to_yolo_format(bbs_aug, margin = bb_margin, width = img_width, height = img_height)
  res = cv2.cvtColor(images_aug[0], cv2.COLOR_RGB2BGR)
  img_file_name = destinationimagePath + '3_'+image_name
  cv2.imwrite(img_file_name, res)
  ann_file_name = destinationannotationPath + '3_'+image_name.replace(image_format, '.txt')
  write_annotation_to_file_augmentation(ih, iw, bbs_aug, ann_file_name)
  print('3', end = ' ')

  images_aug, bbs_aug = aug4(images=image2, bounding_boxes=bbs)
  bbs_aug = convert_bbox_to_yolo_format(bbs_aug, margin = bb_margin, width = img_width, height = img_height)
  res = cv2.cvtColor(images_aug[0], cv2.COLOR_RGB2BGR)
  img_file_name = destinationimagePath + '4_'+image_name
  cv2.imwrite(img_file_name, res)
  ann_file_name = destinationannotationPath + '4_'+image_name.replace(image_format, '.txt')
  write_annotation_to_file_augmentation(ih, iw, bbs_aug, ann_file_name)
  print('4', end = ' ')

  images_aug, bbs_aug = aug5(images=image2, bounding_boxes=bbs)
  bbs_aug = convert_bbox_to_yolo_format(bbs_aug, margin = bb_margin, width = img_width, height = img_height)
  res = cv2.cvtColor(images_aug[0], cv2.COLOR_RGB2BGR)
  img_file_name = destinationimagePath + '5_'+image_name
  cv2.imwrite(img_file_name, res)
  ann_file_name = destinationannotationPath + '5_'+image_name.replace(image_format, '.txt')
  write_annotation_to_file_augmentation(ih, iw, bbs_aug, ann_file_name)
  print('5', end = ' ')

  images_aug, bbs_aug = aug6(images=image2, bounding_boxes=bbs)
  bbs_aug = convert_bbox_to_yolo_format(bbs_aug, margin = bb_margin, width = img_width, height = img_height)
  res = cv2.cvtColor(images_aug[0], cv2.COLOR_RGB2BGR)
  img_file_name = destinationimagePath + '6_'+image_name
  cv2.imwrite(img_file_name, res)
  ann_file_name = destinationannotationPath + '6_'+image_name.replace(image_format, '.txt')
  write_annotation_to_file_augmentation(ih, iw, bbs_aug, ann_file_name)
  print('6', end = ' ')

  images_aug, bbs_aug = aug7(images=image2, bounding_boxes=bbs)
  bbs_aug = convert_bbox_to_yolo_format(bbs_aug, margin = bb_margin, width = img_width, height = img_height)
  res = cv2.cvtColor(images_aug[0], cv2.COLOR_RGB2BGR)
  img_file_name = destinationimagePath + '7_'+image_name
  cv2.imwrite(img_file_name, res)
  ann_file_name = destinationannotationPath + '7_'+image_name.replace(image_format, '.txt')
  write_annotation_to_file_augmentation(ih, iw, bbs_aug, ann_file_name)
  print('7', end = ' ')

  images_aug, bbs_aug = aug8(images=image2, bounding_boxes=bbs)
  bbs_aug = convert_bbox_to_yolo_format(bbs_aug, margin = bb_margin, width = img_width, height = img_height)
  res = cv2.cvtColor(images_aug[0], cv2.COLOR_RGB2BGR)
  img_file_name = destinationimagePath + '8_'+image_name
  cv2.imwrite(img_file_name, res)
  ann_file_name = destinationannotationPath + '8_'+image_name.replace(image_format, '.txt')
  write_annotation_to_file_augmentation(ih, iw, bbs_aug, ann_file_name)
  print('8', end = ' ')

  images_aug, bbs_aug = aug9(images=image2, bounding_boxes=bbs)
  bbs_aug = convert_bbox_to_yolo_format(bbs_aug, margin = bb_margin, width = img_width, height = img_height)
  res = cv2.cvtColor(images_aug[0], cv2.COLOR_RGB2BGR)
  img_file_name = destinationimagePath + '9_'+image_name
  cv2.imwrite(img_file_name, res)
  ann_file_name = destinationannotationPath + '9_'+image_name.replace(image_format, '.txt')
  write_annotation_to_file_augmentation(ih, iw, bbs_aug, ann_file_name)
  print('9', end = ' ')

  images_aug, bbs_aug = aug10(images=image2, bounding_boxes=bbs)
  bbs_aug = convert_bbox_to_yolo_format(bbs_aug, margin = bb_margin, width = img_width, height = img_height)
  res = cv2.cvtColor(images_aug[0], cv2.COLOR_RGB2BGR)
  img_file_name = destinationimagePath + '10_'+image_name
  cv2.imwrite(img_file_name, res)
  ann_file_name = destinationannotationPath + '10_'+image_name.replace(image_format, '.txt')
  write_annotation_to_file_augmentation(ih, iw, bbs_aug, ann_file_name)
  print('10', end = ' ')

  images_aug, bbs_aug = aug11(images=image2, bounding_boxes=bbs)
  bbs_aug = convert_bbox_to_yolo_format(bbs_aug, margin = bb_margin, width = img_width, height = img_height)
  res = cv2.cvtColor(images_aug[0], cv2.COLOR_RGB2BGR)
  img_file_name = destinationimagePath + '11_'+image_name
  cv2.imwrite(img_file_name, res)
  ann_file_name = destinationannotationPath + '11_'+image_name.replace(image_format, '.txt')
  write_annotation_to_file_augmentation(ih, iw, bbs_aug, ann_file_name)
  print('11')

Running augmentation on Trainning images folder on dataset :

In [ ]:
destinationimagePath = '/content/dataset/train/images/'
destinationannotationPath = '/content/dataset/train/labels/'

for i in range(1,151,1):
  image_name = '{0:03}'.format(i) + image_format
  sourceimage = destinationimagePath + image_name
  sourceannotation = '/content/data/annotations/400_senior/' + '{0:03}'.format(i) + '.txt'
  bbs = []
  
  with open(sourceannotation) as myfile:
    for x in range(19):
      content = ''
      line = (next(myfile).strip().split(','))
      line = xy_to_bbox_augmentation(line, margin = augmentation_margin, width = img_width, height = img_height)
      bbs.append(ia.BoundingBox(x1=line[0], y1=line[1], x2=line[2], y2=line[3],label = x))
  
  image = cv2.imread(sourceimage)
  image2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  ih = image.shape[0]
  iw = image.shape[1]
  image2 = image2.reshape(1, ih, iw, 3)

  aug(image2, bbs)


Running augmentation on Testing images folder on dataset :

In [ ]:
destinationimagePath = '/content/dataset/test/images/'
destinationannotationPath = '/content/dataset/test/labels/'

for i in range(151,301,1):
  image_name = '{0:03}'.format(i) + image_format
  sourceimage = destinationimagePath + image_name
  sourceannotation = '/content/data/annotations/400_senior/' + '{0:03}'.format(i) + '.txt'
  bbs = []
  
  with open(sourceannotation) as myfile:
    for x in range(19):
      content = ''
      line = (next(myfile).strip().split(','))
      line = xy_to_bbox_augmentation(line, margin = bb_margin, width = img_width, height = img_height)
      bbs.append(ia.BoundingBox(x1=line[0], y1=line[1], x2=line[2], y2=line[3],label = x))
  
  image = cv2.imread(sourceimage)
  image2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  ih = image.shape[0]
  iw = image.shape[1]
  image2 = image2.reshape(1, ih, iw, 3)

  aug(image2, bbs)


Running augmentation on Validation images folder on dataset :

In [ ]:
destinationimagePath = '/content/dataset/valid/images/'
destinationannotationPath = '/content/dataset/valid/labels/'

for i in range(301,401,1):
  image_name = '{0:03}'.format(i) + image_format
  sourceimage = destinationimagePath + image_name
  sourceannotation = '/content/data/annotations/400_senior/' + '{0:03}'.format(i)+'.txt'
  bbs = []
  
  with open(sourceannotation) as myfile:
    for x in range(19):
      content = ''
      line = (next(myfile).strip().split(','))
      line = xy_to_bbox_augmentation(line, margin = bb_margin, width = img_width, height = img_height)
      bbs.append(ia.BoundingBox(x1=line[0], y1=line[1], x2=line[2], y2=line[3],label = x))
  
  image = cv2.imread(sourceimage)
  image2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  ih = image.shape[0]
  iw = image.shape[1]
  image2 = image2.reshape(1, ih, iw, 3)

  aug(image2, bbs)


----------------------------------------
**Preparing hamedan_university database for yolov5**

Creating standard split folder for train, test and validation :

In [ ]:
!rm -r /content/hamedan_university_dataset
!mkdir /content/hamedan_university_dataset

!mkdir /content/hamedan_university_dataset/test
!mkdir /content/hamedan_university_dataset/train
!mkdir /content/hamedan_university_dataset/valid

!mkdir /content/hamedan_university_dataset/test/images
!mkdir /content/hamedan_university_dataset/test/labels

!mkdir /content/hamedan_university_dataset/train/images
!mkdir /content/hamedan_university_dataset/train/labels

!mkdir /content/hamedan_university_dataset/valid/images
!mkdir /content/hamedan_university_dataset/valid/labels

In [ ]:
training_ratio = 0.8
bb_margin = 25

In [ ]:
import os
images_list = os.listdir('/content/hamedan_university/hamedan_university_dataset/pictures')

In [ ]:
import random
random.shuffle(images_list)

In [ ]:
print(images_list)

In [ ]:
training_image_idx = round(training_ratio * len(images_list))
testing_image_idx = training_image_idx + round(((1 - training_ratio)/2) * len(images_list))
validation_image_idx = len(images_list)
print (training_image_idx, testing_image_idx, validation_image_idx)

In [ ]:
import json
def justify_annotation(file_name, bb_margin = 25):
  with open(file_name, 'r') as f:
    data = json.load(f)
  
  ih = data['imageHeight']
  iw = data['imageWidth']
  annotation = []
  for i in data['shapes']:
    label = i['label'][1::]
    px = i['points'][0][0]
    py = i['points'][0][1]
    annotation.append([int(label)-1, (px/iw), (py/ih), (bb_margin/iw), (bb_margin/ih)])
  return annotation

In [ ]:
def write_annotation_to_file(annotationf, annotationm, file_name):
  fp = open(file_name, 'w')
  for x in range(len(annotationf)):
      #content = ''
      line = annotationf[x]
      content = str(line[0])
      for item in line[1::]:
        content = content + ' ' + str(item)
      # write each item on a new line
      fp.write("%s\n" % content)
  
  for x in range(len(annotationm)):
      #content = ''
      line = annotationm[x]
      content = str(line[0])
      for item in line[1::]:
        content = content + ' ' + str(item)
      # write each item on a new line
      fp.write("%s\n" % content)

In [ ]:
dataset_image_path = "/content/hamedan_university/hamedan_university_dataset/pictures/"
dataset_annotation_f = "/content/hamedan_university/hamedan_university_dataset/annotated by Dr.farhadifar/"
dataset_annotation_m = "/content/hamedan_university/hamedan_university_dataset/annotated by  Dr.mollabashi/"

In [ ]:
from pathlib import Path
import shutil
import cv2
for i in range(0, training_image_idx):
  sourcefile = os.path.join(dataset_image_path, images_list[i])
  destinationfile = '/content/hamedan_university_dataset/train/images/' + images_list[i].replace('.tif', '.jpg')
  annotation_farhadifar = dataset_annotation_f + images_list[i].replace('.tif', '.json')
  annotation_mollabashi = dataset_annotation_m + images_list[i].replace('.tif', '.json')
  img = cv2.imread(sourcefile)
  cv2.imwrite(destinationfile, img)
  file_name = '/content/hamedan_university_dataset/train/labels/' + images_list[i].replace('.tif', '.txt')
  if (Path(annotation_farhadifar).is_file() and Path(annotation_mollabashi).is_file()):
    annotation_m = justify_annotation(annotation_mollabashi);
    annotation_f = justify_annotation(annotation_farhadifar);
    img = cv2.imread(sourcefile)
    cv2.imwrite(destinationfile, img)
    write_annotation_to_file(annotation_f, annotation_m, file_name)
  elif (Path(annotation_farhadifar).is_file() and not Path(annotation_mollabashi).is_file()):
    annotation_f = justify_annotation(annotation_farhadifar);
    img = cv2.imread(sourcefile)
    cv2.imwrite(destinationfile, img)
    write_annotation_to_file(annotation_f, annotation_f, file_name)
  elif (not Path(annotation_farhadifar).is_file() and Path(annotation_mollabashi).is_file()):
    annotation_m = justify_annotation(annotation_mollabashi);
    img = cv2.imread(sourcefile)
    cv2.imwrite(destinationfile, img)
    write_annotation_to_file(annotation_m, annotation_m, file_name)

for i in range(training_image_idx, testing_image_idx):
  sourcefile = os.path.join(dataset_image_path, images_list[i])
  destinationfile = '/content/hamedan_university_dataset/test/images/' + images_list[i].replace('.tif', '.jpg')
  annotation_farhadifar = dataset_annotation_f + images_list[i].replace('.tif', '.json')
  annotation_mollabashi = dataset_annotation_m + images_list[i].replace('.tif', '.json')
  img = cv2.imread(sourcefile)
  cv2.imwrite(destinationfile, img)
  file_name = '/content/hamedan_university_dataset/test/labels/' + images_list[i].replace('.tif', '.txt')
  if (Path(annotation_farhadifar).is_file() and Path(annotation_mollabashi).is_file()):
    annotation_m = justify_annotation(annotation_mollabashi);
    annotation_f = justify_annotation(annotation_farhadifar);
    img = cv2.imread(sourcefile)
    cv2.imwrite(destinationfile, img)
    write_annotation_to_file(annotation_f, annotation_m, file_name)
  elif (Path(annotation_farhadifar).is_file() and not Path(annotation_mollabashi).is_file()):
    annotation_f = justify_annotation(annotation_farhadifar);
    img = cv2.imread(sourcefile)
    cv2.imwrite(destinationfile, img)
    write_annotation_to_file(annotation_f, annotation_f, file_name)
  elif (not Path(annotation_farhadifar).is_file() and Path(annotation_mollabashi).is_file()):
    annotation_m = justify_annotation(annotation_mollabashi);
    img = cv2.imread(sourcefile)
    cv2.imwrite(destinationfile, img)
    write_annotation_to_file(annotation_m, annotation_m, file_name)

for i in range(testing_image_idx, len(images_list)):
  sourcefile = os.path.join(dataset_image_path, images_list[i])
  destinationfile = '/content/hamedan_university_dataset/valid/images/' + images_list[i].replace('.tif', '.jpg')
  annotation_farhadifar = dataset_annotation_f + images_list[i].replace('.tif', '.json')
  annotation_mollabashi = dataset_annotation_m + images_list[i].replace('.tif', '.json')
  img = cv2.imread(sourcefile)
  cv2.imwrite(destinationfile, img)
  file_name = '/content/hamedan_university_dataset/valid/labels/' + images_list[i].replace('.tif', '.txt')
  if (Path(annotation_farhadifar).is_file() and Path(annotation_mollabashi).is_file()):
    annotation_m = justify_annotation(annotation_mollabashi);
    annotation_f = justify_annotation(annotation_farhadifar);
    img = cv2.imread(sourcefile)
    cv2.imwrite(destinationfile, img)
    write_annotation_to_file(annotation_f, annotation_m, file_name)
  elif (Path(annotation_farhadifar).is_file() and not Path(annotation_mollabashi).is_file()):
    annotation_f = justify_annotation(annotation_farhadifar);
    img = cv2.imread(sourcefile)
    cv2.imwrite(destinationfile, img)
    write_annotation_to_file(annotation_f, annotation_f, file_name)
  elif (not Path(annotation_farhadifar).is_file() and Path(annotation_mollabashi).is_file()):
    annotation_m = justify_annotation(annotation_mollabashi);
    img = cv2.imread(sourcefile)
    cv2.imwrite(destinationfile, img)
    write_annotation_to_file(annotation_m, annotation_m, file_name)


--------------------------------

# **Yolo5 :**

cloning yolo repository :

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

**On yolov5 filder:**

*   yolov5s_result  : before augmentation, train on ISBI
*   yolov5s_result2 : after augmentation, train on ISBI
*   yolov5s_result3 : fine tune on hamedan_university_dataset






In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat ../dataset/data.yaml

In [ ]:
# define number of classes based on YAML
import yaml
with open("../dataset/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
print(num_classes)

In [ ]:
#this is the model configuration we will use for our tutorial 
%cat /content/yolov5/models/yolov5s.yaml

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

**yolo5s :**

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

**yolo5x :**

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5x.yaml

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 1.33  # model depth multiple
width_multiple: 1.25  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

**Train :**

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/

# YoloV5s
!python train.py --img 416 --batch 16 --epochs 500 --data /content/dataset/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

# YoloV5x
#!python train.py --img 416 --batch 16 --epochs 500 --data /content/dataset/data.yaml --cfg ./models/custom_yolov5x.yaml --weights '' --name yolov5x_results  --cache

------------------------------

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason... 
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5/runs/train/yolov5s_results/results.png', width=1000)  # view results.png

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/

**Evaluation (ISBI) :**

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
# use the best weights!
%cd /content/yolov5/

# before augmentation
#!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.3 --source ../dataset/test/images --iou-thres 0.0 --save-txt --hide-conf #--max-det 19

# after augmentation
#!python detect.py --weights runs/train/yolov5s_results2/weights/best.pt --img 416 --conf 0.3 --source ../dataset/test/images --iou-thres 0.0 --save-txt --hide-conf #--max-det 19

# after fine tune
!python detect.py --weights runs/train/yolov5s_results3/weights/best.pt --img 416 --conf 0.3 --source ../dataset/test/images --iou-thres 0.0 --save-txt --hide-conf #--max-det 19

------------------------------------------
**show grand truth on test images (ISBI):**

In [ ]:
!rm -r /content/result
!mkdir /content/result

In [ ]:
# ISBI
img_width = 1935
img_height = 2400

image_format = '.jpg'

In [ ]:
test_images_path = '/content/yolov5/runs/detect/exp'
test_labels_path = '/content/dataset/test/labels'
result_path = '/content/result/'

In [ ]:
import os
import cv2

images_list = os.listdir(test_images_path)
for idx in range(len(images_list)):
  image_name = test_images_path + '/' + images_list[idx]
  label_name = test_labels_path + '/' + images_list[idx].replace(image_format,'.txt')
  result_name =  result_path + images_list[idx].replace(image_format,'.jpg')
  print(f'{images_list[idx]:<15}', end = '-')
  print("%4d of %4d - %6.2f%%" %((idx+1), len(images_list), ((idx + 1)/len(images_list) * 100)))
  image = cv2.imread(image_name)
  with open(label_name) as labelfile:
    for x in range(19):
      line = (next(labelfile).strip().split(' '))
      x_value = int(float(line[1]) * img_width)
      y_value = int(float(line[2]) * img_height)
      cv2.circle(image, (x_value, y_value), 5, (0, 0, 0), 10)
      cv2.putText(image, str(x+1), (x_value-15, y_value-15), cv2.FONT_HERSHEY_SIMPLEX , 1,(255, 0, 0), 2, cv2.LINE_AA)
      cv2.imwrite(result_name, image)
print('done')

  

**Evaluation (hamedan_university):**

In [ ]:
# when we ran this, we saw .007 second inference time. That is 140 FPS on a TESLA P100!
# use the best weights!
%cd /content/yolov5/

# before augmentation
#!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.3 --source ../hamedan_university_dataset/test/images --iou-thres 0.0 --save-txt --hide-conf #--max-det 19

# after augmentation
#!python detect.py --weights runs/train/yolov5s_results2/weights/best.pt --img 416 --conf 0.3 --source ../hamedan_university_dataset/test/images --iou-thres 0.0 --save-txt --hide-conf #--max-det 19

# after fine tune
!python detect.py --weights runs/train/yolov5s_results3/weights/best.pt --img 416 --conf 0.3 --source ../hamedan_university_dataset/test/images --iou-thres 0.0 --save-txt --hide-conf #--max-det 19

------------------------------------------
**show grand truth on test images (hamedan_university):**

In [ ]:
!rm -r /content/result
!mkdir /content/result

In [ ]:
# hamedan_university
img_width = 1340
img_height = 1671

image_format = '.jpg'

In [ ]:
test_images_path = '/content/yolov5/runs/detect/exp'
test_labels_path = '/content/hamedan_university_dataset/test/labels'
result_path = '/content/result/'

In [ ]:
import os
import cv2

images_list = os.listdir(test_images_path)
for idx in range(len(images_list)):
  image_name = test_images_path + '/' + images_list[idx]
  label_name = test_labels_path + '/' + images_list[idx].replace(image_format,'.txt')
  result_name =  result_path + images_list[idx].replace(image_format,'.jpg')
  print(f'{images_list[idx]:<15}', end = '-')
  print("%4d of %4d - %6.2f%%" %((idx+1), len(images_list), ((idx + 1)/len(images_list) * 100)))
  image = cv2.imread(image_name)
  cv2.putText(image, 'farhadifar', (50, 50), cv2.FONT_HERSHEY_SIMPLEX , 1, (255, 0, 0), 1, cv2.LINE_AA)
  cv2.putText(image, 'molabashi', (50, 100), cv2.FONT_HERSHEY_SIMPLEX , 1, (0, 255, 0), 1, cv2.LINE_AA)
  with open(label_name) as labelfile:
    for x in range(19):
      line = (next(labelfile).strip().split(' '))
      x_value = int(float(line[1]) * img_width)
      y_value = int(float(line[2]) * img_height)
      cv2.circle(image, (x_value, y_value), 3, (255, 0, 0), 3)
      cv2.putText(image, str((x%19)+1), (x_value-15, y_value-15), cv2.FONT_HERSHEY_SIMPLEX , 1,(255, 0, 0), 1, cv2.LINE_AA)
      cv2.imwrite(result_name, image)
    for x in range(19, 38):
      line = (next(labelfile).strip().split(' '))
      x_value = int(float(line[1]) * img_width)
      y_value = int(float(line[2]) * img_height)
      cv2.circle(image, (x_value, y_value), 3, (0, 255, 0), 3)
      cv2.putText(image, str((x%19)+1), (x_value-15, y_value-15), cv2.FONT_HERSHEY_SIMPLEX , 1,(0, 255, 0), 1, cv2.LINE_AA)
      cv2.imwrite(result_name, image)
print('done')

  

**Calculating Overlap area (hamedan_university):**

In [ ]:
detection_path = '/content/yolov5/runs/detect/exp/labels'
label_path = '/content/hamedan_university_dataset/test/labels'
img_width = 1340
img_height = 1671
margin = 25

In [ ]:
import math
def distance(a, b):  # returns 0 if rectangles don't intersect
    return (math.dist([a.x, a.y], [b.x, b.y]))

**Farhadifar:**

In [ ]:
import os
import csv

from collections import namedtuple
Point = namedtuple('Point', 'x y')

csvfile = open('/content/detection_farhadifar.csv', 'w')
headeres = ['image_name', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', \
              'L10', 'L11', 'L12', 'L13', 'L14', 'L15', 'L16', 'L17', 'L18', 'L19']
writer = csv.writer(csvfile)
writer.writerow(headeres)

detection_list = os.listdir(detection_path)
for idx in range(len(detection_list)):
  cls_overlap_area = [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
  file_name = detection_list[idx]
  detection_name = detection_path + '/' + file_name
  label_name = label_path + '/' + file_name

  label_file = open(label_name, 'r')
  label_Lines = label_file.readlines()

  with open(detection_name) as detection_file:
    detection_Lines = detection_file.readlines()
    for detection_line in detection_Lines:
      detection_line = detection_line.split(' ')
      detection_cls = int(detection_line[0])
      detection_x = int(float(detection_line[1]) * img_width)
      detection_y = int(float(detection_line[2]) * img_height)

      label_line = label_Lines[detection_cls].split(' ')
      label_x = int(float(label_line[1]) * img_width)
      label_y = int(float(label_line[2]) * img_height)

      cls_overlap_area[0] = int(file_name.split('.')[0])

      pa = Point(detection_x, detection_y)
      pb = Point(label_x, label_y)
      cls_overlap_area[detection_cls + 1] = distance(pa, pb)

  writer.writerow(cls_overlap_area)
csvfile.close()
print('done')

**Molabashi:**

In [ ]:
import os
import csv

from collections import namedtuple
Point = namedtuple('Point', 'x y')

csvfile = open('/content/detection_molabashi.csv', 'w')
headeres = ['image_name', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', \
              'L10', 'L11', 'L12', 'L13', 'L14', 'L15', 'L16', 'L17', 'L18', 'L19']
writer = csv.writer(csvfile)
writer.writerow(headeres)

detection_list = os.listdir(detection_path)
for idx in range(len(detection_list)):
  cls_overlap_area = [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
  file_name = detection_list[idx]
  detection_name = detection_path + '/' + file_name
  label_name = label_path + '/' + file_name

  label_file = open(label_name, 'r')
  label_Lines = label_file.readlines()

  with open(detection_name) as detection_file:
    detection_Lines = detection_file.readlines()
    for detection_line in detection_Lines:
      detection_line = detection_line.split(' ')
      detection_cls = int(detection_line[0])
      detection_x = int(float(detection_line[1]) * img_width)
      detection_y = int(float(detection_line[2]) * img_height)

      label_line = label_Lines[detection_cls+19].split(' ')
      # label_cls = int(label_line[0])
      label_x = int(float(label_line[1]) * img_width)
      label_y = int(float(label_line[2]) * img_height)

      cls_overlap_area[0] = int(file_name.split('.')[0])

      pa = Point(detection_x, detection_y)
      pb = Point(label_x, label_y)
      cls_overlap_area[detection_cls + 1] = distance(pa, pb)

  writer.writerow(cls_overlap_area)
csvfile.close()
print('done')